Con el fin de cumplir con el trabajo pedido por el equipo de data analyts se realizará el etl correspondiente a los 4 datasets que me fueron entregados; en este cuaderno iré comentando cada una de las sentencias de forma detallada para que sea más facil de identificar algún error, hacer alguna correción o usar parte del codigo en otro análisis.

In [1]:
#Importamos las librerias a utilizar 
import pandas as pd
import numpy as np

In [2]:
#Cargamos los datasets 
amazon = pd.read_csv('amazon_prime_titles-score.csv')
disney = pd.read_csv('disney_plus_titles-score.csv')
hulu = pd.read_csv('hulu_titles-score.csv')
netflix = pd.read_csv('netflix_titles-score.csv')

* En primera instancia se trabajará con el dataframe de amazon 

In [3]:
#vizualizamos los datos 
amazon.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,99
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,37
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,20
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",27
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,75


In [4]:
#Tamaño del dataframe de amazon
print("Columnas:", str(amazon.shape[1]))
print("filas: ", str(amazon.shape[0]))

Columnas: 13
filas:  9668


In [5]:
#Vemos si hay duplicados 
amazon[amazon.duplicated()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score


In [6]:
#Observamos la cantidad de valores faltantes por columna
amazon.isna().sum()

show_id            0
type               0
title              0
director        2082
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
score              0
dtype: int64

Como no se nos ha pedido nada acerca de estos valores faltantes los dejaremos asi nada mas

In [7]:
amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
 12  score         9668 non-null   int64 
dtypes: int64(2), object(11)
memory usage: 982.0+ KB


                                                    Transformaciones

* Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

In [8]:
#Creare una función que agregue la letra de la plataforma amazon al comienzo  del id

def agregar_letra_a(id):
    return 'a' + str(id)

In [9]:
#Probamos que la función sirva 
numero = 123
agregar_letra_a(numero)

'a123'

In [10]:
#aplicamos la función agregar_letra_a a la columna show_id de amazon
amazon['show_id'] = amazon['show_id'].apply(agregar_letra_a)

In [11]:
#vizualizamos la columna show_id para comprobar que se haya aplicado bien la funcion a los campos de esta columna
amazon['show_id']

0          as1
1          as2
2          as3
3          as4
4          as5
         ...  
9663    as9664
9664    as9665
9665    as9666
9666    as9667
9667    as9668
Name: show_id, Length: 9668, dtype: object

* Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

In [12]:
#Usamos el metodo fillna para reemplazar los valores nulos por la letra G
amazon['rating'].fillna('G', inplace=True)

In [13]:
amazon['rating'].unique()

array(['G', '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'PG',
       'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE'], dtype=object)

* De haber fechas, deberán tener el formato AAAA-mm-dd

In [14]:
#primero convertimos la columna fechas al formato datetime
amazon['date_added'] = pd.to_datetime(amazon['date_added'])

In [15]:
#vizualizamos y vemos que ya esta de la forma AAAA-mm-dd, damos este punto por cumplido
amazon

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,as1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,2021-03-30,2014,G,113 min,"Comedy, Drama",A small fishing village must procure a local d...,99
1,as2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,2021-03-30,2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,37
2,as3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,2021-03-30,2017,G,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,20
3,as4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,2021-03-30,2014,G,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",27
4,as5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,2021-03-30,1989,G,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9663,as9664,Movie,Pride Of The Bowery,Joseph H. Lewis,"Leo Gorcey, Bobby Jordan",NaN,NaT,1940,7+,60 min,Comedy,New York City street principles get an East Si...,7
9664,as9665,TV Show,Planet Patrol,NaN,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",NaN,NaT,2018,13+,4 Seasons,TV Shows,"This is Earth, 2100AD - and these are the adve...",49
9665,as9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaT,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou...",68
9666,as9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaT,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ...",72


* El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [20]:
#Se crearan las dos columnas que se pidieron, extraeremos y separaremos los datos usando el metodo str.extract

amazon['duration_int'] = amazon['duration'].str.extract('(\d+)', expand=False)
amazon['duration_type'] = amazon['duration'].str.extract('([A-Za-z]+)', expand=False)

* Los campos de texto deberán estar en minúsculas, sin excepciones

In [23]:
#Usamos el método lower para pasar todo a minusculas en las columnas con texto
amazon['type'] = amazon['type'].str.lower()
amazon['title'] = amazon['title'].str.lower()
amazon['director'] = amazon['director'].str.lower()
amazon['cast'] = amazon['cast'].str.lower()
amazon['country'] = amazon['country'].str.lower()
amazon['listed_in'] = amazon['listed_in'].str.lower()
amazon['description'] = amazon['description'].str.lower()


In [24]:
amazon.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,duration_int,duration_type
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,G,113 min,"comedy, drama",a small fishing village must procure a local d...,99,113,min
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,37,110,min
2,as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,G,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...,20,74,min
3,as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,G,69 min,documentary,"pink breaks the mold once again, bringing her ...",27,69,min
4,as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,G,45 min,"drama, fantasy",teenage matt banting wants to work with a famo...,75,45,min


* En segunda instancia haremos la limpieza al daframe de la plataforma disney plus

In [16]:
#Vizualizamos como viene la data 
disney.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!,100
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...,17
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.,92
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!",35
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...,11


In [25]:
#informacion
disney.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
 12  score         1450 non-null   int64 
dtypes: int64(2), object(11)
memory usage: 147.4+ KB


In [26]:
disney.isna().sum()

show_id           0
type              0
title             0
director        473
cast            190
country         219
date_added        3
release_year      0
rating            3
duration          0
listed_in         0
description       0
score             0
dtype: int64

In [27]:
#chequeamos que no existan duplicados 
disney[disney.duplicated()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score


ya que las transformaciones sugeridas no dice nada acerca de la imputación paso inmediatamente hacer las transformaciones correspondientes

* 1) Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

In [28]:
#creamos una funcion que agrege la letra d i la concatene con el id del dataframe de disney
def agregar_letra_d(id):
    return 'd' + str(id)


In [30]:
#probamos funcion
texto = '12as'
agregar_letra_d(texto)

'd12as'

In [31]:
#Ahora usamos la función creada para el dataframe disney

disney['show_id'] = disney['show_id'].apply(agregar_letra_d)

In [32]:
#comprobamos que se haya concatenado la letra de en la columna show_id
disney['show_id'].unique()

array(['ds1', 'ds2', 'ds3', ..., 'ds1448', 'ds1449', 'ds1450'],
      dtype=object)

* Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

In [33]:
#Usamos el metodo fillna en la columna rating

disney['rating'].fillna('G', inplace=True)

* De haber fechas, deberán tener el formato AAAA-mm-dd

In [34]:
#usamos la funcion to_datetime para cambiar el formato de fecha 
disney['date_added'] = pd.to_datetime(disney['date_added'])

In [35]:
#Visualizamos como queda, y ya esta en el formato deseado
disney.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,ds1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,2021-11-26,2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!,100
1,ds2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,2021-11-26,1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...,17
2,ds3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,2021-11-26,2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.,92
3,ds4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,2021-11-26,2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!",35
4,ds5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,2021-11-25,2021,G,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...,11


* Los campos de texto deberán estar en minúsculas, sin excepciones

In [36]:
#usamos el metodo lower en las columnas que tengan datos en texto
disney['type'] = disney['type'].str.lower()
disney['title'] = disney['title'].str.lower()
disney['director'] = disney['director'].str.lower()
disney['cast'] = disney['cast'].str.lower()
disney['country'] = disney['country'].str.lower()
disney['listed_in'] = disney['listed_in'].str.lower()
disney['description'] = disney['description'].str.lower()

In [37]:
#verificamos que quedaron los campos de texto en minusculas
disney.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,ds1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",NaN,2021-11-26,2016,TV-G,23 min,"animation, family",join mickey and the gang as they duck the halls!,100
1,ds2,movie,ernest saves christmas,john cherry,"jim varney, noelle parker, douglas seale",NaN,2021-11-26,1988,PG,91 min,comedy,santa claus passes his magic bag to a new st. ...,17
2,ds3,movie,ice age: a mammoth christmas,karen disher,"raymond albert romano, john leguizamo, denis l...",united states,2021-11-26,2011,TV-G,23 min,"animation, comedy, family",sid the sloth is on santa's naughty list.,92
3,ds4,movie,the queen family singalong,hamish hamilton,"darren criss, adam lambert, derek hough, alexa...",NaN,2021-11-26,2021,TV-PG,41 min,musical,"this is real life, not just fantasy!",35
4,ds5,tv show,the beatles: get back,NaN,"john lennon, paul mccartney, george harrison, ...",NaN,2021-11-25,2021,G,1 Season,"docuseries, historical, music",a three-part documentary from peter jackson ca...,11


* El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [38]:

disney['duration_int'] = disney['duration'].str.extract('(\d+)', expand=False)
disney['duration_type'] = disney['duration'].str.extract('([A-Za-z]+)', expand=False)

In [39]:
#Verificamos que se extrajo y separo la infomacion de la columna duration a dos columnas nuevas
disney.tail()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,duration_int,duration_type
1445,ds1446,movie,x-men origins: wolverine,gavin hood,"hugh jackman, liev schreiber, danny huston, wi...","united states, united kingdom",2021-06-04,2009,PG-13,108 min,"action-adventure, family, science fiction",wolverine unites with legendary x-men to fight...,53,108,min
1446,ds1447,movie,night at the museum: battle of the smithsonian,shawn levy,"ben stiller, amy adams, owen wilson, hank azar...","united states, canada",2021-04-02,2009,PG,106 min,"action-adventure, comedy, family",larry daley returns to rescue some old friends...,75,106,min
1447,ds1448,movie,eddie the eagle,dexter fletcher,"tom costello, jo hartley, keith allen, dickon ...","united kingdom, germany, united states",2020-12-18,2016,PG-13,107 min,"biographical, comedy, drama","true story of eddie edwards, a british ski-jum...",89,107,min
1448,ds1449,movie,bend it like beckham,gurinder chadha,"parminder nagra, keira knightley, jonathan rhy...","united kingdom, germany, united states",2020-09-18,2003,PG-13,112 min,"buddy, comedy, coming of age",despite the wishes of their traditional famili...,77,112,min
1449,ds1450,movie,captain sparky vs. the flying saucers,mark waring,charlie tahan,united states,2020-04-01,2012,TV-G,2 min,"action-adventure, animals & nature, animation",view one of sparky's favorite home movies.,88,2,min


* En tercera instancia se haran las transformaciones al dataframe hulu

In [41]:
#visualizamos como viene la data
hulu.head(50)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...,48
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r...",8
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...,8
3,s4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...,33
4,s5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,NaN,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...,24
5,s6,TV Show,The Halloween Candy Magic Pet,NaN,NaN,NaN,"October 22, 2021",2021,NaN,1 Season,"Family, Kids",Join Mila and Morphle on a mystery-filled Hall...,45
6,s7,Movie,The Evil Next Door,NaN,NaN,NaN,"October 21, 2021",2020,NaN,88 min,"Horror, Thriller","New to her role as a stepmom, a young woman mo...",37
7,s8,TV Show,The Next Thing You Eat,NaN,NaN,NaN,"October 21, 2021",2021,NaN,1 Season,"Cooking & Food, Documentaries, Lifestyle & Cul...",With the unique insights and experience of Ugl...,76
8,s9,TV Show,Queens,NaN,NaN,NaN,"October 20, 2021",2021,TV-14,1 Season,"Drama, Music",Four women in their 40s reunite for a chance t...,57
9,s10,TV Show,The Bachelorette,NaN,NaN,United States,"October 20, 2021",2003,TV-14,3 Seasons,"Reality, Romance",ABC's romance reality show lets one lucky lady...,79


In [42]:
#Chequeamos cuantos nulos hay en este dataframe, solo haremos cambio a estos nulos en la columna rating
hulu.isna().sum()

show_id            0
type               0
title              0
director        3070
cast            3073
country         1453
date_added        28
release_year       0
rating           520
duration         479
listed_in          0
description        4
score              0
dtype: int64

In [43]:
#informacion del dataframe hulu
hulu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
 12  score         3073 non-null   int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 312.2+ KB


                                                Comenzamos con las transformaciones

* Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

In [44]:
#creamos una funcion que concatene al id de hulu la letra h

def agregar_letra_h(id):
    return 'h' + str(id)

In [45]:
#Comprobamos por descarte para ver si cumple su cometido la funcion creada
varchar = '12as'
agregar_letra_h(varchar)

'h12as'

In [46]:
#Ya que la funcion corre bien la aplicamos a la columna show_id del dataframe hulu
hulu['show_id'] = hulu['show_id'].apply(agregar_letra_h)

In [47]:
#Comprobamos
hulu['show_id'].unique()

array(['hs1', 'hs2', 'hs3', ..., 'hs3071', 'hs3072', 'hs3073'],
      dtype=object)

* Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

In [48]:
#Usamos el metodo fillna para complir con la consigna anterior
hulu['rating'].fillna('G', inplace=True)

In [56]:
hulu.rating.unique()

array(['TV-MA', 'G', 'PG-13', 'R', 'TV-14', 'PG', 'TV-PG', 'NOT RATED',
       'TV-G', '2 Seasons', 'TV-Y', '93 min', '4 Seasons', 'TV-Y7',
       '136 min', '91 min', '85 min', '98 min', '89 min', '94 min',
       '86 min', '3 Seasons', '121 min', '88 min', '101 min', '1 Season',
       '83 min', '100 min', '95 min', '92 min', '96 min', '109 min',
       '99 min', '75 min', '87 min', '67 min', '104 min', '107 min',
       '84 min', '103 min', '105 min', '119 min', '114 min', '82 min',
       '90 min', '130 min', '110 min', '80 min', '6 Seasons', '97 min',
       '111 min', '81 min', '49 min', '45 min', '41 min', '73 min',
       '40 min', '36 min', '39 min', '34 min', '47 min', '65 min',
       '37 min', '78 min', '102 min', '129 min', '115 min', '112 min',
       'NR', '61 min', '106 min', '76 min', '77 min', '79 min', '157 min',
       '28 min', '64 min', '7 min', '5 min', '6 min', '127 min',
       '142 min', '108 min', '57 min', '118 min', '116 min', '12 Seasons',
       '71 min']

* De haber fechas, deberán tener el formato AAAA-mm-dd

In [57]:
#Usamos la funcion to_datetime para cambiar el formato de fechas
hulu['date_added'] = pd.to_datetime(hulu['date_added'])

In [58]:
hulu.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,hs1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,2021-10-24,2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...,48
1,hs2,Movie,Silent Night,NaN,NaN,NaN,2021-10-23,2020,G,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r...",8
2,hs3,Movie,The Marksman,NaN,NaN,NaN,2021-10-23,2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...,8
3,hs4,Movie,Gaia,NaN,NaN,NaN,2021-10-22,2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...,33
4,hs5,Movie,Settlers,NaN,NaN,NaN,2021-10-22,2021,G,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...,24


* Los campos de texto deberán estar en minúsculas, sin excepciones

In [59]:
#Cambiamos los campos de texto a solo minusculas
hulu['type'] = hulu['type'].str.lower()
hulu['title'] = hulu['title'].str.lower()
hulu['director'] = hulu['director'].str.lower()
hulu['country'] = hulu['country'].str.lower()
hulu['listed_in'] = hulu['listed_in'].str.lower()
hulu['description'] = hulu['description'].str.lower()

In [60]:
#Verificamos que se hayan cambiado
hulu.tail()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
3068,hs3069,tv show,star trek: the original series,NaN,NaN,united states,NaT,1966,TV-PG,3 Seasons,"action, adventure, classics",the 23rd century adventures of captain james t...,14
3069,hs3070,tv show,star trek: voyager,NaN,NaN,united states,NaT,1997,TV-PG,7 Seasons,"action, adventure, science fiction",catapulted into the distant sector of the gala...,53
3070,hs3071,tv show,the fades,NaN,NaN,united kingdom,NaT,2011,TV-14,1 Season,"horror, international, science fiction",seventeen-year-old paul is haunted by apocalyp...,81
3071,hs3072,tv show,the twilight zone,NaN,NaN,united states,NaT,1959,TV-PG,5 Seasons,"classics, science fiction, thriller",rod serling's seminal anthology series focused...,81
3072,hs3073,tv show,tokyo magnitude 8.0,NaN,NaN,japan,NaT,2009,TV-14,1 Season,"anime, drama, international",the devastation is unleashed in the span of se...,47


* El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [61]:
#creamos las dos columnas pedidas por la consigna y ademas usamos el metodo extract para extraer la informacion separada para cada columna
hulu['duration_int'] = hulu['duration'].str.extract('(\d+)', expand=False)
hulu['duration_type'] = hulu['duration'].str.extract('([A-Za-z]+)', expand=False)

In [62]:
hulu.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,duration_int,duration_type
0,hs1,movie,ricky velez: here's everything,NaN,NaN,NaN,2021-10-24,2021,TV-MA,NaN,"comedy, stand up",​comedian ricky velez bares it all with his ho...,48,NaN,NaN
1,hs2,movie,silent night,NaN,NaN,NaN,2021-10-23,2020,G,94 min,"crime, drama, thriller","mark, a low end south london hitman recently r...",8,94,min
2,hs3,movie,the marksman,NaN,NaN,NaN,2021-10-23,2021,PG-13,108 min,"action, thriller",a hardened arizona rancher tries to protect an...,8,108,min
3,hs4,movie,gaia,NaN,NaN,NaN,2021-10-22,2021,R,97 min,horror,a forest ranger and two survivalists with a cu...,33,97,min
4,hs5,movie,settlers,NaN,NaN,NaN,2021-10-22,2021,G,104 min,"science fiction, thriller",mankind's earliest settlers on the martian fro...,24,104,min


* En esta cuarta y ultima instacia de transformaciones trabajaremos con el dataframe netflix

In [63]:
#vizualizamos como viene la data
netflix.head(50)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",29
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",86
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,75
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",49
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,92
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,37
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,12
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...",10
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...,44
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...,3


In [65]:
#informacion
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
 12  score         8807 non-null   int64 
dtypes: int64(2), object(11)
memory usage: 894.6+ KB


In [66]:
netflix.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
score              0
dtype: int64

Pasamos directamente a las transformaciones propuestas 

* Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

In [67]:
#creamos una funcion que concatene la letra n a los valores de la columna show_id del dataframe netflix

def agregar_letra_n(id):
    return 'n' + str(id)

In [68]:
#Provamos antes si sirve la funcón creada :P

string = "peli"
agregar_letra_n(string)

'npeli'

In [69]:
#como la funcion sirve la aplicamos a la columna show_id del dataframe netflix

netflix['show_id'] = netflix['show_id'].apply(agregar_letra_n)

In [72]:
#verificamos que se haya agregado la letra n a los valores de la columna show_id
netflix.show_id.unique()

array(['ns1', 'ns2', 'ns3', ..., 'ns8805', 'ns8806', 'ns8807'],
      dtype=object)

* Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

In [73]:
#Usamos el metodo fillna para agregar la letra G en los campos nulos de la columna rating
netflix['rating'].fillna('G', inplace=True)

* De haber fechas, deberán tener el formato AAAA-mm-dd

In [74]:
#usamos el metodo to_datetime de pandas para cambiar el formato de fechas
netflix['date_added'] = pd.to_datetime(netflix['date_added'])

In [76]:
#verificamos
netflix.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,ns1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",29


* Los campos de texto deberán estar en minúsculas, sin excepciones

In [77]:
#Usamos el metodo lower para cambiar los campos de texto del dataframe nexflix a minusculas

netflix['type'] = netflix['type'].str.lower()
netflix['title'] = netflix['title'].str.lower()
netflix['director'] = netflix['director'].str.lower()
netflix['cast'] = netflix['cast'].str.lower()
netflix['country'] = netflix['country'].str.lower()
netflix['listed_in'] = netflix['listed_in'].str.lower()
netflix['description'] = netflix['description'].str.lower()


* El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [78]:
##creamos las dos columnas pedidas por la consigna y ademas usamos el metodo extract para extraer la informacion separada para cada columna
netflix['duration_int'] = netflix['duration'].str.extract('(\d+)', expand=False)
netflix['duration_type'] = netflix['duration'].str.extract('([A-Za-z]+)', expand=False)

In [79]:
#chequeamos como queda el dataframe nexflix luego de las transformaciones
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,duration_int,duration_type
0,ns1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,PG-13,90 min,documentaries,"as her father nears the end of his life, filmm...",29,90,min
1,ns2,tv show,blood & water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,2021-09-24,2021,TV-MA,2 Seasons,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t...",86,2,Seasons
2,ns3,tv show,ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"crime tv shows, international tv shows, tv act...",to protect his family from a powerful drug lor...,75,1,Season
3,ns4,tv show,jailbirds new orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,1 Season,"docuseries, reality tv","feuds, flirtations and toilet talk go down amo...",49,1,Season
4,ns5,tv show,kota factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,2021-09-24,2021,TV-MA,2 Seasons,"international tv shows, romantic tv shows, tv ...",in a city of coaching centers known to train i...,92,2,Seasons


Ya tengo los dataframe con las transformaciones pedidas por el proyecto, ahora creare un solo dataframe que contenga la información de cada plataforma de peliculas

Ahora solo vamos concatenando cada dataframe al nuevo dataframe data_plataforma

In [99]:
data_plataforma = pd.concat([amazon, disney, hulu, netflix], axis=0)

In [100]:
data_plataforma 

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,duration_int,duration_type
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,G,113 min,"comedy, drama",a small fishing village must procure a local d...,99,113,min
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,37,110,min
2,as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,G,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...,20,74,min
3,as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,G,69 min,documentary,"pink breaks the mold once again, bringing her ...",27,69,min
4,as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,G,45 min,"drama, fantasy",teenage matt banting wants to work with a famo...,75,45,min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,ns8803,movie,zodiac,david fincher,"mark ruffalo, jake gyllenhaal, robert downey j...",united states,2019-11-20,2007,R,158 min,"cult movies, dramas, thrillers","a political cartoonist, a crime reporter and a...",20,158,min
8803,ns8804,tv show,zombie dumb,NaN,NaN,NaN,2019-07-01,2018,TV-Y7,2 Seasons,"kids' tv, korean tv shows, tv comedies","while living alone in a spooky town, a young g...",8,2,Seasons
8804,ns8805,movie,zombieland,ruben fleischer,"jesse eisenberg, woody harrelson, emma stone, ...",united states,2019-11-01,2009,R,88 min,"comedies, horror movies",looking to survive in a world taken over by zo...,55,88,min
8805,ns8806,movie,zoom,peter hewitt,"tim allen, courteney cox, chevy chase, kate ma...",united states,2020-01-11,2006,PG,88 min,"children & family movies, comedies","dragged from civilian life, a former superhero...",7,88,min


ya con el nuevo dataframe unificado creamos un nuevo csv para trabajar con las consultas pero esto lo haremos en otro cuaderno en forma de prueba para cuando ya cremos la api sabremos que funcion ejecutar para cada consulta

In [101]:
#generamos un nuevo csv 
data_plataforma.to_csv('data_plataformas.csv', index=False)